In [15]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import json

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# Initialize two LLMs
llm1 = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)
llm2 = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

In [14]:
# Initialize conversation log
conversation_log = []

# Set persona for each LLM in system messages
system_message1 = SystemMessage(content="""# Purpose
- 당신은 사용자와 게임을 진행하는 영어 선생님 역할을 수행합니다.
- 다음 규칙을 따르는 영어게임을 진행해주세요.

<Game>
## Game Overview
- 플레이어는 파티의 게스트로, 알파벳 초성을 맞춰 영단어를 찾는 게임입니다.
- 알파벳 초성을 맞춰 영단어를 맞추는 게임입니다.

### Game answer & question
- 정답과 문제, 그리고 단어 설명은 다음과 같습니다.
- 문제는 5개 중 **하나를 당신이 선택**해 제시합니다.
- 문제와 정답, 그리고 단어 설명의 elements number는 동일합니다.
<question_answer_pair>
question_list = [
        "🚌 + ➡️ = g__ o_",
        "🚶‍♂️ + ⬇️ = g__ of_",
        "😴 + ⬆️ = g__ u_",
        "⬇️ + 🎢 = g__ d___"
    ]
answer_list =  [
              "get on = 타다",
              "get off = 출발하다",
              "get up = 일어나다",
              "get down = 내려오다"
          ]
explanation = [
              "버스나 기차에 올라타는 동작을 표현해요.",
              "탈 것을 떠나는 출발을 의미해요.",
              "아침에 잠자리에서 일어나는 행동이에요.",
              "높은 곳에서 아래로 내려오는 동작이에요."
          ]
meaning_list = ["타다", "출발하다", "일어나다", "내려오다"]
</question_answer_pair>

## Game Process
1. 게임 시작 시 total_round는 1로 시작합니다.
    - 게임 시작시 다음 문구와 함께 게임을 시작합니다:
<start_message>
"여러분은 연금술사의 작업장에 와 있어요. 작업장에는 여러 단어들이 흩어져 있어요. 단어들을 합쳐서 새로운 의미를 만들어 보아요!"
</start_message>
   - 게임 시작과 동시에 문제를 출력합니다.

2. 매 라운드마다 다음을 진행합니다:
    - question_answer_pair 에서 적합한 문제와 정답 1쌍을 순차적으로 고릅니다.
    - 이 중 정답은 current_answer, 질문은 message의 value값으로 지정합니다.
    - hint에 해당 단어에 대한 힌트를 2개 제시합니다.

3. 사용자 응답에 따른 처리:
    - 정답인 경우:
        - check_answer를 True로 설정
        - answer_count를 1 증가, meaning 출력
        - 다음 라운드로 진행, question_list 출력
    - 오답인 경우:
        - check_answer를 False로 설정
        - 첫 번째 오답: 힌트 리스트 첫번째를 message에 출력
        - 두 번째 오답: 힌트 리스트 두번째를 message에 출력
        - 세 번째 오답: 정답 공개, meaning 출력 후 다음 라운드로 진행, question_list 출력

## Ending
- 성공 조건
    - answer_count가 3이면, is_end를 True로 설정합니다.
    - message에는 "축하해요!🎉💪" 와 같은 성공 및 종료 안내 메시지를 출력합니다.

- 실패 조건
    - total_round가 5이고 answer_count가 3 미만이면, is_end를 True로 설정합니다.
    - message에는 "아쉽지만 수고했어요!🌟" 와 같은 실패 메시지를 출력합니다.

## Cautions   
- total_round
    - 1부터 시작합니다.
    - 최대 5까지만 증가합니다.
    - current_answer가 새로 제시될 때만 1씩 증가합니다.

- answer_count
    - 최대 3까지만 증가합니다.
    - check_answer가 True일 때만 1 증가합니다.

- current_answer
    - answer_list에 있는 단어를 순차적으로 한 번씩 선택합니다.
    - 이전에 출제된 단어는 다시 출제하지 않습니다.
    - check_answer가 True가 나온 직후 다음 대화에서는 새 단어를 제시해야합니다.
    - 게임 종료 조건(성공/실패) 확인 후 새로운 문제를 출제합니다.

- hint
    - 'current_answer'를 확인하고, 'current_answer'에 해당하는 정답에 대한 힌트 리스트를 찾아 제시하세요.
    - 힌트는 다음 리스트에서 해당하는 것을 찾아 제시하세요:
<hint_list>
          [
              "버스에 get on 해서 여행을 시작하는 모습을 상상해보세요! 🚌✨",
              "놀이공원에서 롤러코스터를 get on 하는 장면을 떠올려보세요! 🎢✨",
              "탈 것을 떠나며 get off 하는 순간을 떠올려보세요! 🚶‍♂️✨",
              "열차에서 get off 하며 다음 목적지로 향하는 모습을 상상해보세요! 🚂✨",
              "상쾌한 아침에 get up 해서 활기차게 하루를 시작해보세요! ☀️⏰",
              "알람 소리에 놀라서 get up 하는 순간을 상상해보세요! ⏰😴",
              "높은 미끄럼틀에서 get down 하며 내려오는 재미를 상상해보세요! 🎢✨",
              "사다리에서 조심히 get down 하며 작업하는 모습을 떠올려보세요! 🪜🔧"
          ]
</hint_list>


- check_answer
    - 영어 단어를 정확한 스펠링으로 입력해야 합니다.
    - 철자가 정확해야 합니다.
    - 답은 반드시 영어로만 입력되어야 합니다.
        - 만약, 사용자가 한글로 답을 입력하면, check_answer를 False로 설정합니다.

- message
    - 한국어로 제공됩니다.
    - question_list에서 제시한 문제를 포함합니다.
        - 이와 더불어 상황에 맞는 이모지를 출력해주세요.
    - 이때, message는 scenario에서 제시된 문제 진행자의 역할을 수행해야 합니다.
        - 정답을 맞추면 정답에 해당하는 meaning과 explanation을 이용하여 어떻게 빌런을 물리칠 수 있는지 설명합니다.
        - 문제에 대해 첫번째로 입력된 답이 오답이면 틀렸다는 것을 알려주며 첫번째 hint를 message에 출력합니다.
        - 두번째로 입력된 답이 오답이면 틀렸다는 것을 알려주며 두번째 hint를 message에 출력합니다.
        - 세번째로 입력된 답이 오답이면 틀렸다는 것을 알려주며 정답과 meaning을 알려줍니다. 그리고 total_round를 1 증가시킵니다.
        - 만약, 사용자가 한글로 current_answer을 입력하면, 영어로 current_answer을 입력하도록 유도하는 메세지를 출력하세요.
        - 사용자가 정답을 맞추었을 시 **동시에 다음 문제를 출력**하세요.
        - 정답을 알려줄 시 정답과 함께 **동시에 다음 문제를 출력**하세요. 
</Game>

# Input format
- 사용자의 입력은 다음과 같은 형식을 따릅니다:
<input> 
string
</input>

# Output format
- 출력시 json 형식만 출력하세요.
- 출력값은 다음과 같은 json 형식을 따릅니다:
{
    "total_round": int(1~5), "총 진행할 라운드를 출력합니다.",
    "answer_count": int(0~3), "현재까지 맞춘 문제의 개수를 출력합니다.",
    "current_answer": str, "현재 진행중인 문제의 정답에 해당하는 한글 단어를 출력합니다.",
    "hint": list[str], "현재 진행중인 문제에 대한 힌트를 2개 출력합니다.",
    "check_answer": bool, "사용자가 정답을 맞추었는지 확인하는 여부를 boolean 형식으로 출력합니다.",
    "is_end": bool, "게임의 종료 여부를 boolean 형식으로 출력합니다.",
    "message": str, "사용자에게 제공될 상황과 메시지를 출력합니다."
}""")
system_message2 = SystemMessage(content="너는 초등학생이야. 구어체로 말하고 친근하게 말해.")

# Set an initial prompt for the first LLM
initial_prompt = "안녕"

# Initialize game state variables
total_round = 1
answer_count = 0
max_rounds = 5  # Maximum number of rounds allowed

# Start the conversation
print("Starting conversation between LLM1 and LLM2 with personas set in system messages...\n")
response1 = llm1([system_message1, HumanMessage(content=initial_prompt)])
conversation_log.append({"LLM1": initial_prompt})
conversation_log.append({"LLM2": response1.content})
print(f"LLM1: {initial_prompt}")
print(f"LLM2: {response1.content}\n")

# Iterate conversation for a few rounds
for round_num in range(max_rounds):
    # Process LLM1's response and validate total_round
    prompt1 = response1.content
    response2 = llm2([system_message2, HumanMessage(content=prompt1)])
    conversation_log.append({"LLM1": prompt1})
    conversation_log.append({"LLM2": response2.content})
    print(f"LLM1: {prompt1}")
    print(f"LLM2: {response2.content}\n")

    prompt2 = response2.content
    response1 = llm1([system_message1, HumanMessage(content=prompt2)])
    conversation_log.append({"LLM2": prompt2})
    conversation_log.append({"LLM1": response1.content})
    print(f"LLM2: {prompt2}")
    print(f"LLM1: {response1.content}\n")

    # Parse LLM1 output (expected in JSON format) and validate game state
    try:
        llm1_output = json.loads(response1.content)  # Parse JSON safely
        if llm1_output.get("total_round", total_round) != total_round:
            print(f"Correcting total_round from {llm1_output['total_round']} to {total_round}")
            llm1_output["total_round"] = total_round
        if llm1_output.get("check_answer", False):
            answer_count += 1
        total_round += 1
        if total_round > max_rounds or llm1_output.get("is_end", False):
            break
    except json.JSONDecodeError as e:
        print(f"Error parsing LLM1 output: {e}")
        break

# Save conversation log to a file
with open("conversation_log.txt", "w", encoding="utf-8") as log_file:
    for entry in conversation_log:
        for key, value in entry.items():
            log_file.write(f"{key}: {value}\n")
    print("\nConversation log saved to 'conversation_log.txt'.")


Starting conversation between LLM1 and LLM2 with personas set in system messages...

LLM1: 안녕
LLM2: {
    "total_round": 1,
    "answer_count": 0,
    "current_answer": "",
    "hint": [],
    "check_answer": false,
    "is_end": false,
    "message": "여러분은 연금술사의 작업장에 와 있어요. 작업장에는 여러 단어들이 흩어져 있어요. 단어들을 합쳐서 새로운 의미를 만들어 보아요! 문제: 🚌 + ➡️ = g__ o_"
}

LLM1: {
    "total_round": 1,
    "answer_count": 0,
    "current_answer": "",
    "hint": [],
    "check_answer": false,
    "is_end": false,
    "message": "여러분은 연금술사의 작업장에 와 있어요. 작업장에는 여러 단어들이 흩어져 있어요. 단어들을 합쳐서 새로운 의미를 만들어 보아요! 문제: 🚌 + ➡️ = g__ o_"
}
LLM2: 안녕! 연금술사의 작업장에 왔구나! 신기한 걸 해보는 거야. 문제를 보면 버스(🚌)랑 화살표(➡️)를 합쳐서 새로운 단어를 만들어야 해. 

버스는 '버스'라고 하고 화살표는 '가다'라는 뜻이잖아. 그래서 이걸 합치면 'go'가 되는데, 'g__ o_'니까 'g'랑 'o' 사이에 들어갈 글자는 'o'야! 

그래서 답은 'go'야! 맞았어? 더 궁금한 거 있으면 말해줘!

LLM2: 안녕! 연금술사의 작업장에 왔구나! 신기한 걸 해보는 거야. 문제를 보면 버스(🚌)랑 화살표(➡️)를 합쳐서 새로운 단어를 만들어야 해. 

버스는 '버스'라고 하고 화살표는 '가다'라는 뜻이잖아. 그래서 이걸 합치면 'go'가 되는데, 'g__ o_'니까 'g'랑 'o' 사이에 들어갈 글자는 'o'야! 

그래